In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, round, max, avg, sum, array, count, dense_rank
from pyspark.sql.functions import to_timestamp, date_format, from_utc_timestamp, hour, min, year, month
from pyspark.sql.types import FloatType
import requests
from pyspark.sql.window import Window
from haversine import haversine, Unit

In [3]:
spark = SparkSession.builder.appName("projectFirst").getOrCreate()

your 131072x1 screen size is bogus. expect trouble
23/09/06 11:35:15 WARN Utils: Your hostname, SUSHAN resolves to a loopback address: 127.0.1.1; using 172.31.68.109 instead (on interface eth0)
23/09/06 11:35:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/06 11:35:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
fuel_prices_df = spark.read.parquet("data/cleaned_fuel_prices.parquet")
station_info_df = spark.read.parquet("data/cleaned_station_info.parquet")

In [5]:
def convert_to_euro(usd_price):
    # API URL to fetch the exchange rate
    api_url = "https://cdn.jsdelivr.net/gh/fawazahmed0/currency-api@1/latest/currencies/usd/eur.json"
    
    # Fetch exchange rate data from the API
    response = requests.get(api_url)
    exchange_rate_data = response.json()
    
    # Extract the exchange rate
    exchange_rate = exchange_rate_data["eur"]
    
    # Convert USD to Euro
    euro_price = usd_price * exchange_rate
    return euro_price


In [6]:
# Apply the UDF to convert prices from USD to Euro
fuel_prices_df = fuel_prices_df.withColumn("Price_Euro", convert_to_euro(col("Price")))
# Round the Euro prices to three decimal units
fuel_prices_df = fuel_prices_df.withColumn("Price_Euro_Rounded", round(col("Price_Euro"), 3))

# Drop the "Price_Euro" column
fuel_prices_df = fuel_prices_df.drop("Price_Euro")

# Show the updated DataFrame
fuel_prices_df.show()

+-----+------+-----+-------------------+------------------+
|   Id|isSelf|Price|               Date|Price_Euro_Rounded|
+-----+------+-----+-------------------+------------------+
| 5079|     1|1.809|2022-01-04 06:39:09|             1.676|
|38752|     1|1.804|2022-01-04 07:18:19|             1.671|
|51635|     0|1.974|2022-01-04 07:39:23|             1.828|
| 6810|     0|2.009|2022-01-04 07:50:04|             1.861|
| 4983|     0|1.758|2022-01-04 07:55:36|             1.628|
|51790|     1|1.819|2022-01-04 08:50:47|             1.685|
|42708|     0|2.064|2022-01-04 08:51:50|             1.912|
|48545|     1|1.799|2022-01-04 09:26:35|             1.666|
|46455|     0|1.954|2022-01-04 12:14:28|              1.81|
|47555|     0|2.048|2022-01-04 12:58:37|             1.897|
| 6711|     1|1.819|2022-01-04 13:15:06|             1.685|
|31878|     1|1.784|2022-01-04 13:15:25|             1.652|
|52822|     1|1.769|2022-01-04 13:19:01|             1.639|
|51870|     0|2.054|2022-01-04 13:40:13|

In [7]:
# Find the maximum price
max_price = fuel_prices_df.agg(max("Price")).collect()[0][0]

# Find the ID with the highest price
highest_price_id = fuel_prices_df.filter(col("Price") == max_price).select("Id").first()[0]

# Order the data by price in descending order
sorted_prices_df = fuel_prices_df.orderBy(col("Price").desc())

# Calculate the average price
average_price = fuel_prices_df.agg(avg("Price")).collect()[0][0]

# Display the results
print("ID with Highest Price:", highest_price_id)
print("Maximum Average Price:", average_price)

ID with Highest Price: 54771
Maximum Average Price: 1.835294686861716


In [8]:
fuel_prices_df = fuel_prices_df.withColumn("Day_of_Week", date_format(col("Date"), "E"))

day_sales_df = fuel_prices_df.groupBy("Day_of_Week").agg(count("*").alias("Sales_Count"))

# Order the data by sales count in descending order to rank the days
window_spec = Window.orderBy(col("Sales_Count").desc())
day_sales_ranked_df = day_sales_df.withColumn("Rank", dense_rank().over(window_spec))

day_sales_ranked_df.show()

23/09/06 11:35:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/06 11:35:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/06 11:35:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/06 11:35:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/06 11:35:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/09/06 11:35:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+-----------+-----------+----+
|Day_of_Week|Sales_Count|Rank|
+-----------+-----------+----+
|        Mon|     410910|   1|
|        Thu|     385288|   2|
|        Wed|     375737|   3|
|        Tue|     363014|   4|
|        Fri|     362752|   5|
|        Sat|     308091|   6|
|        Sun|     238545|   7|
+-----------+-----------+----+



In [9]:
station_info_df_copy = station_info_df.select(
    [col(col_name).alias(col_name + "_d2") for col_name in station_info_df.columns]
)

df_joined = station_info_df.crossJoin(station_info_df_copy)

In [10]:
# Convert string columns to float using the `cast` function
df_joined = df_joined.withColumn("Latitude", df_joined["Latitude"].cast(FloatType()))
df_joined = df_joined.withColumn("Longitudine", df_joined["Longitudine"].cast(FloatType()))
df_joined = df_joined.withColumn("Latitude_d2", df_joined["Latitude_d2"].cast(FloatType()))
df_joined = df_joined.withColumn("Longitudine_d2", df_joined["Longitudine_d2"].cast(FloatType()))

In [11]:
print(df_joined.printSchema())

root
 |-- Id: string (nullable = true)
 |-- Petrol_company: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Station_name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Latitude: float (nullable = true)
 |-- Longitudine: float (nullable = true)
 |-- Id_d2: string (nullable = true)
 |-- Petrol_company_d2: string (nullable = true)
 |-- Type_d2: string (nullable = true)
 |-- Station_name_d2: string (nullable = true)
 |-- City_d2: string (nullable = true)
 |-- Latitude_d2: float (nullable = true)
 |-- Longitudine_d2: float (nullable = true)

None


In [12]:
# Define a user-defined function (UDF) to calculate distances
@udf(FloatType())
def calculate_distance(lat1, lon1, lat2, lon2):
    return haversine((lat1, lon1), (lat2, lon2), unit=Unit.KILOMETERS)

# Add a new column 'Distance_between' to the DataFrame using the UDF
result_df = df_joined.withColumn("Distance_between", calculate_distance(
    col("Latitude"), col("Longitudine"), col("Latitude_d2"), col("Longitudine_d2")
))

# Select the desired columns
result_df = result_df.select("Id", "Petrol_company", "Type", "Station_name", "City", "Id_d2", "Petrol_company_d2", "Type_d2", "Station_name_d2", "City_d2", "Distance_between")

# Show the resulting DataFrame
result_df.show(truncate=False)

+-----+--------------+--------+--------------------------------+----------+-----+-----------------+--------+-------------------------------------------------------------------+------------------------+----------------+
|Id   |Petrol_company|Type    |Station_name                    |City      |Id_d2|Petrol_company_d2|Type_d2 |Station_name_d2                                                    |City_d2                 |Distance_between|
+-----+--------------+--------+--------------------------------+----------+-----+-----------------+--------+-------------------------------------------------------------------+------------------------+----------------+
|11461|Q8            |Stradale|AYMAVILLES-F.NE VILLETOS S.R. 47|AYMAVILLES|11461|Q8               |Stradale|AYMAVILLES-F.NE VILLETOS S.R. 47                                   |AYMAVILLES              |0.0             |
|11461|Q8            |Stradale|AYMAVILLES-F.NE VILLETOS S.R. 47|AYMAVILLES|35245|Agip Eni         |Stradale|Castel di Lama C

In [13]:
# Filter the DataFrame to select stations within 100 kilometers
filtered_df = result_df.filter(result_df["Distance_between"] < 100)

# Show the filtered DataFrame
filtered_df.show(truncate=False)

23/09/06 11:36:19 WARN ExtractPythonUDFFromJoinCondition: The join condition:(calculate_distance(cast(Latitude#13 as float), cast(Longitudine#14 as float), cast(Latitude_d2#140 as float), cast(Longitudine_d2#141 as float))#232 < 100.0) of the join plan contains PythonUDF only, it will be moved out and the join plan will be turned to cross join.


+-----+--------------+--------+--------------------------------+----------+-----+-----------------+--------+------------------------------------+------------------+----------------+
|Id   |Petrol_company|Type    |Station_name                    |City      |Id_d2|Petrol_company_d2|Type_d2 |Station_name_d2                     |City_d2           |Distance_between|
+-----+--------------+--------+--------------------------------+----------+-----+-----------------+--------+------------------------------------+------------------+----------------+
|11461|Q8            |Stradale|AYMAVILLES-F.NE VILLETOS S.R. 47|AYMAVILLES|11461|Q8               |Stradale|AYMAVILLES-F.NE VILLETOS S.R. 47    |AYMAVILLES        |0.0             |
|11461|Q8            |Stradale|AYMAVILLES-F.NE VILLETOS S.R. 47|AYMAVILLES|29453|Api-Ip           |Stradale|TotalErg                            |CHIERI            |88.12894        |
|11461|Q8            |Stradale|AYMAVILLES-F.NE VILLETOS S.R. 47|AYMAVILLES|31773|Pompe Bia

In [14]:
# Extract the day of the week from the 'Date' column and create a new column 'Day_of_Week'
fuel_prices_df = fuel_prices_df.withColumn("Day_of_Week", date_format(col("Date"), "E"))

# Pivot the table to get counts for each day of the week
pivot_df = fuel_prices_df.groupBy().pivot("Day_of_Week").agg(count("*"))

# Display the result
pivot_df.show()

+------+------+------+------+------+------+------+
|   Fri|   Mon|   Sat|   Sun|   Thu|   Tue|   Wed|
+------+------+------+------+------+------+------+
|362752|410910|308091|238545|385288|363014|375737|
+------+------+------+------+------+------+------+



In [15]:
merged_df = fuel_prices_df.join(station_info_df , on="Id", how="inner")

merged_df.show()

+-----+------+-----+-------------------+------------------+-----------+--------------+------------+--------------------+-------------------+-----------------+------------------+
|   Id|isSelf|Price|               Date|Price_Euro_Rounded|Day_of_Week|Petrol_company|        Type|        Station_name|               City|         Latitude|       Longitudine|
+-----+------+-----+-------------------+------------------+-----------+--------------+------------+--------------------+-------------------+-----------------+------------------+
| 5079|     1|1.809|2022-01-04 06:39:09|             1.676|        Tue|      Agip Eni|    Stradale|_eni settebagni c...|               ROMA| 42.0108885470618| 12.51839518547058|
|38752|     1|1.804|2022-01-04 07:18:19|             1.671|        Tue|      Agip Eni|    Stradale|          eni pv1351|             SAVONA|44.29415084326569| 8.452990427613258|
|51635|     0|1.974|2022-01-04 07:39:23|             1.828|        Tue|      Agip Eni|    Stradale|   ENI STAT

In [16]:
# Extract the hour from the "Date" timestamp and add it as a new column "Hour"
merged_df = merged_df.withColumn("Hour", hour(merged_df["Date"]))

# Filter the data to include only self-service fuel stations in specific cities (Calcinate, Osio Sopra, Treviglio, Biella) and with Petrol_company Q8
cities_to_include = ["AGRIGENTO", "OSIO SOPRA"]
filtered_df = merged_df.filter((merged_df["isSelf"] == 1) & (merged_df["City"].isin(cities_to_include)) & (merged_df["Petrol_company"] == "Q8"))

print("Filtered Data Count:", filtered_df.count())

Filtered Data Count: 625


In [17]:
# Define a window specification to partition data by 'Petrol_company' and 'Hour'
window_spec = Window.partitionBy('Petrol_company', 'Hour')

# Calculate the total revenue for each group
total_revenue_df = filtered_df.withColumn('Total_Revenue', sum('Price').over(window_spec))

# Display the resulting DataFrame
total_revenue_df.show()

+-----+------+-----+-------------------+------------------+-----------+--------------+--------+--------------------+---------+-----------------+------------------+----+-----------------+
|   Id|isSelf|Price|               Date|Price_Euro_Rounded|Day_of_Week|Petrol_company|    Type|        Station_name|     City|         Latitude|       Longitudine|Hour|    Total_Revenue|
+-----+------+-----+-------------------+------------------+-----------+--------------+--------+--------------------+---------+-----------------+------------------+----+-----------------+
|28277|     1|1.739|2022-04-04 00:35:12|             1.611|        Mon|            Q8|Stradale|QUADRIVIO SPINANS...|AGRIGENTO|37.31919829622775|13.581069525760311|   0|91.20899999999999|
|28277|     1|2.019|2022-06-27 00:37:14|              1.87|        Mon|            Q8|Stradale|QUADRIVIO SPINANS...|AGRIGENTO|37.31919829622775|13.581069525760311|   0|91.20899999999999|
|50463|     1|2.089|2022-06-27 00:37:14|             1.935|      

In [18]:
# Convert the timestamp to the specified timezone (America/New_York) and extract the hour
hour_df = merged_df.withColumn("Hour", hour(from_utc_timestamp(merged_df["Date"], "America/New_York")))

# Calculate hourly price fluctuations for each city
price_fluctuations_df = hour_df.groupBy("City", "Hour").agg((max("Price") - min("Price")).alias("Price_Fluctuation"))

# Calculate max and min prices separately
max_prices_df = hour_df.groupBy("City", "Hour").agg(max("Price").alias("Max_Price"))
min_prices_df = hour_df.groupBy("City", "Hour").agg(min("Price").alias("Min_Price"))

# Join max and min prices together
fluctuation_with_max_min_df = price_fluctuations_df.join(max_prices_df, ["City", "Hour"], "inner").join(min_prices_df, ["City", "Hour"], "inner")

# Create an array column combining max and min prices
price_array_df = fluctuation_with_max_min_df.withColumn("Prices", array("Max_Price", "Min_Price"))

# Find the top 10 cities with the highest price fluctuations
top_cities_fluctuations = price_array_df.groupBy("City").agg(avg("Price_Fluctuation").alias("Avg_Fluctuation")).orderBy("Avg_Fluctuation", ascending=False).limit(10)

# Join the DataFrames on the "City" column
result_df = top_cities_fluctuations.join(price_array_df, ["City"], "inner")

# Select the desired columns and order by "Avg_Fluctuation" in descending order
result_df = result_df.select("Prices", "Avg_Fluctuation", "City").distinct().orderBy(result_df["Avg_Fluctuation"].desc())

# Show the result
result_df.show(truncate=False)

+--------------+------------------+-------------------+
|Prices        |Avg_Fluctuation   |City               |
+--------------+------------------+-------------------+
|[1.639, 1.639]|1.5020909090909091|SAN DEMETRIO CORONE|
|[4.0, 1.639]  |1.5020909090909091|SAN DEMETRIO CORONE|
|[4.0, 1.669]  |1.5020909090909091|SAN DEMETRIO CORONE|
|[4.0, 4.0]    |1.5020909090909091|SAN DEMETRIO CORONE|
|[4.0, 1.789]  |1.5020909090909091|SAN DEMETRIO CORONE|
|[4.0, 1.699]  |1.5020909090909091|SAN DEMETRIO CORONE|
|[1.955, 1.629]|1.5020909090909091|SAN DEMETRIO CORONE|
|[4.0, 1.679]  |1.5020909090909091|SAN DEMETRIO CORONE|
|[4.0, 1.659]  |1.5020909090909091|SAN DEMETRIO CORONE|
|[2.484, 1.587]|1.4498260869565218|TARANTO            |
|[2.497, 1.559]|1.4498260869565218|TARANTO            |
|[2.399, 1.569]|1.4498260869565218|TARANTO            |
|[2.417, 1.547]|1.4498260869565218|TARANTO            |
|[2.358, 1.556]|1.4498260869565218|TARANTO            |
|[2.417, 1.599]|1.4498260869565218|TARANTO      

In [19]:
# Define the window specification for the rolling average
rolling_window_spec = Window.partitionBy("City").orderBy("Date").rowsBetween(-6, 0)

# Calculate the rolling 7-hour average price
rolling_avg_price = filtered_df.withColumn("7_Hour_Avg_Price", avg("Price").over(rolling_window_spec))

# Determine the row with the maximum 7-hour average price within each city
max_avg_price_row = rolling_avg_price.withColumn("max_7_Hour_Avg_Price", max("7_Hour_Avg_Price").over(Window.partitionBy("City")))
result_df = max_avg_price_row.filter(col("7_Hour_Avg_Price") == col("max_7_Hour_Avg_Price")).drop("max_7_Hour_Avg_Price")

# Show the result
result_df.show()

+-----+------+-----+-------------------+------------------+-----------+--------------+------------+------------+----------+-----------------+------------------+----+------------------+
|   Id|isSelf|Price|               Date|Price_Euro_Rounded|Day_of_Week|Petrol_company|        Type|Station_name|      City|         Latitude|       Longitudine|Hour|  7_Hour_Avg_Price|
+-----+------+-----+-------------------+------------------+-----------+--------------+------------+------------+----------+-----------------+------------------+----+------------------+
|49460|     1|2.319|2022-10-06 12:38:53|             2.148|        Thu|            Q8|    Stradale|       AG023| AGRIGENTO|37.32612049037331|13.591820001602168|  12|2.0947142857142858|
|25613|     1|1.839|2022-07-04 14:30:07|             1.703|        Mon|            Q8|Autostradale|  BREMBO SUD|OSIO SOPRA|45.63188736814508| 9.600096659217913|  14|2.0618571428571433|
|25613|     1|2.099|2022-07-06 14:30:45|             1.944|        Wed|    

In [20]:
# Define the specific 'fuel station type' and 'city' to analyze
selected_fuel_station_type = 'Stradale'
selected_city = 'AGRIGENTO' 
# Filter 'fuel_station_df' for the specific combination
filtered_fuel_station_df = station_info_df.filter((col('Type') == selected_fuel_station_type) & (col('City') == selected_city))

# Extract the month from the 'Date' column
joined_df = merged_df.withColumn('YearMonth', year(col('Date')) * 100 + month(col('Date')))

# Group by 'YearMonth' and find the minimum timestamp and minimum price within each group
min_date_price_per_month = joined_df.groupBy('YearMonth').agg(
    min('Date').cast('timestamp').alias('LowestRecordTimestamp'),
    min('Price').alias('LowestPrice')
)

# Find the month with the minimum timestamp and its corresponding lowest price
lowest_timestamp_month = min_date_price_per_month.orderBy('LowestRecordTimestamp').first()

# Display the result (if a lowest timestamp month is found)
if lowest_timestamp_month:
    print(f"The date with the lowest price for '{selected_fuel_station_type}' fuel stations in '{selected_city}' is: {lowest_timestamp_month['LowestRecordTimestamp']}")
    print(f"The lowest price for that month is: {lowest_timestamp_month['LowestPrice']}")
else:
    print(f"No data found for '{selected_fuel_station_type}' fuel stations in '{selected_city}'.")

The date with the lowest price for 'Stradale' fuel stations in 'AGRIGENTO' is: 2022-01-01 11:45:53
The lowest price for that month is: 0.8


In [21]:
# Stop the SparkSession when you're done
spark.stop()